In [ ]:
# Notebook for debugging the pipline

In [3]:
import os
from annabell_utilities import AnnabellLogfileInterpreter
from pipeline import Pipeline
import pandas as pd
import logging

logging.disable(logging.CRITICAL)

base_path_linux = "/home/chris/gdrive"
base_path_mac = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD"
experiment_name = "49"
#json_path = base_path_linux + "/work/annabell/experiments/31/data_sets/nyc_squad_with_pretraining_commands.jsonl"

json_path = base_path_mac + f"/work/annabell/experiments/{experiment_name}/data_sets/nyc_squad_with_pretraining_commands.jsonl"
#json_path = base_path_mac + f"/work/annabell/experiments/{experiment_name}/data_sets/squad_dataset_with_generated_sentences.jsonl"

#json_path = '/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/46/data_sets/nyc_squad_with_pretraining_flags.jsonl'

In [2]:

pipeline = Pipeline(prepared_dataset_filepath=json_path)
pipeline.load_prepared_dataset(json_path)

#pipeline.run_pre_training_evaluation_testing()
#pipeline.run_evaluate_pre_training_results()
#pipeline.generate_pre_training_data()

log_dir = f"/home/chris/gdrive/work/annabell/experiments/{experiment_name}/logs"
log_filename = "annabell_pre_training_log.txt"
log_filepath = os.path.join(log_dir, log_filename)
log_filepath = f"/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/{experiment_name}/logs/annabell_pre_training_log.txt"
interpreter = AnnabellLogfileInterpreter(log_filepath)
interpreter.parse_entries()

2026-02-26 07:11:26,771 - root - INFO - Logging initialized. Log file: /Users/chris/logs/cognitive_language_model_logs/run_20260226_071126.log


FileNotFoundError: [Errno 2] No such file or directory: '/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/49/logs/annabell_pre_training_log.txt'

In [ ]:
#create a graph of entry time vs index
plt = interpreter.plot_entry_time_vs_sample_number()
plt.show()

In [ ]:
#plot a graph showing each stat measure vs index
for plt in interpreter.plot_stat_measures_vs_index():
    plt.show()

In [ ]:
#plot a graph showing "test_answer_correct" vs index

def plot_test_answer_correct(the_df):
    plt.plot(the_df.index, the_df["test_answer_correct"])
    plt.xlabel("Index")
    plt.ylabel("Answer Correct")
    plt.title("Test Answer Correct vs Index")
    plt.show()


df_from_results = pd.read_csv(
    f"/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/{experiment_name}/pre_training/results/test_detailed_results_tsv",
    sep='\t')
plot_test_answer_correct(df_from_results)

In [19]:
#sum the number of distinct words in all of the pre-training commands in the dataset
def count_distinct_words_in_pre_training_commands(the_dataset, up_to_index):
    distinct_words = set()
    for index, row in the_dataset.iterrows():
        if index > up_to_index:
            break
        dec_statement = row["declarative_statement_formatted"]
        question = row["question_formatted"]
        d_words = dec_statement.split()
        q_words = question.split()
        words = d_words + q_words
        distinct_words.update(words)
    return len(distinct_words)


num_distinct_words = count_distinct_words_in_pre_training_commands(df_from_results, up_to_index=268)
print(f"Number of distinct words in pre-training commands: {num_distinct_words}")

Number of distinct words in pre-training commands: 1598


In [ ]:
#plot a graph showing sum of "test_answer_correct" == false vs index

def plot_test_answer_correct(the_df):
    plt.plot(the_df.index, the_df["test_answer_correct"].apply(lambda x: 0 if x else 1).cumsum())
    plt.xlabel("Index")
    plt.ylabel("Answer Not Correct")
    plt.title("Test Answer Not Correct vs Index")
    plt.show()


plot_test_answer_correct(df_from_results)

In [ ]:
    #using the logfile intepreter, plot a graph of the time taken for each pre-training sample vs the count of the sample
import matplotlib.pyplot as plt


def plot_time_taken_vs_sample_count(the_interpreter):
    sample_counts = []
    time_taken = []
    for entry in the_interpreter.entries:
        sample_counts.append(entry.sample_number())
        time_taken.append(entry.time())
    plt.plot(sample_counts, time_taken)
    plt.xlabel("Pre-training Sample Count")
    plt.ylabel("Time Taken (seconds)")
    plt.title("Time Taken vs Pre-training Sample Count")
    plt.show()


plot_time_taken_vs_sample_count(interpreter)


In [15]:
#find the row with id = "56d4eb762ccc5a1400d8334f"
dataframe = pipeline.datasetPreProcessor.dataset
dataframe[dataframe[
              'id'] == "570a826e6d058f1900182eeb"]

,id,title,context,question,declarative_statement,answer,declarative_statement_formatted,question_formatted,answer_formatted,question_category,statement_category
164,570a826e6d058f1900182eeb,Houston,Houston is the seat of the internationally ren...,How many non profit organizations are members ...,The Medical Center has 49 Non-Profit Organizat...,49,the Medical Center have 49 Non - Profit Organi...,? how many non profit organization be member o...,49,NO CATEGORY ASSIGNED,NO CATEGORY ASSIGNED


In [16]:
pre_1000_entry = interpreter.entries[184]
print(
    f"Sample number: {pre_1000_entry.sample_number()}, Learned Words: {pre_1000_entry.learned_words()}, Time: {pre_1000_entry.time()} seconds")
post_1000_entry = interpreter.entries[189]
print(
    f"Sample number: {post_1000_entry.sample_number()}, Learned Words: {post_1000_entry.learned_words()}, Time: {post_1000_entry.time()} seconds")

Sample number: 185, Learned Words: 981, Time: 13.25336199999947 seconds
Sample number: 190, Learned Words: 1000, Time: 18.794857000000775 seconds


In [10]:
dataframe = pipeline.datasetPreProcessor.dataset
#take a random sample of 5 rows form the dataset
indices = [1000, 2000, 3000, 4000, 5000]

examples = []
for index in indices:
    row = dataframe.iloc[index]
    prompt_example = f"EXAMPLE:\n\n\"question\":\"{row["question"]}\", \"answer\":\"{row["answer"]}\"\n\nResponse:\n\n\"\""
    examples.append(prompt_example)
examples


['EXAMPLE:\n\n"question":"What is the official language of Montana?", "answer":"English"\n\nResponse:\n\n""',
 'EXAMPLE:\n\n"question":"What year did the 5th Dalai lama start to dominate over Tibet?", "answer":"1642"\n\nResponse:\n\n""',
 'EXAMPLE:\n\n"question":"Where did a cargo train derail as a result?", "answer":"Hui County, Gansu"\n\nResponse:\n\n""',
 'EXAMPLE:\n\n"question":"What New York street is associated with fashion?", "answer":"Seventh Avenue"\n\nResponse:\n\n""',
 'EXAMPLE:\n\n"question":"What type of language is Portuguese?", "answer":"Romance"\n\nResponse:\n\n""']

In [36]:
commands = ['#id: 57338724d058e614000b5c9f',
            'before adopt national standard Notre_Dame emphasize scholastic and classical', 'education',
            '#END OF DECLARATION', '\n', '? what type of education be push at Notre_Dame',
            'before its embracing of national standard', '#END OF QUESTION',
            '.sctx ? what type of education be push at Notre_Dame', '.pg education', '.pg Notre_Dame',
            '.sctx before its embracing of national standard', '.wg national standard', '.ggp',
            '.ph before adopt national standard Notre_Dame emphasize scholastic and classical', '.drop_goal',
            '.drop_goal', '.wg scholastic and classical', '.rw', '\n', '#END OF COMMANDS', '.time', '#END OF TIME',
            '.stat', '#END OF STATS']
for command in commands:
    print(command)

#id: 57338724d058e614000b5c9f
before adopt national standard Notre_Dame emphasize scholastic and classical
education
#END OF DECLARATION


? what type of education be push at Notre_Dame
before its embracing of national standard
#END OF QUESTION
.sctx ? what type of education be push at Notre_Dame
.pg education
.pg Notre_Dame
.sctx before its embracing of national standard
.wg national standard
.ggp
.ph before adopt national standard Notre_Dame emphasize scholastic and classical
.drop_goal
.drop_goal
.wg scholastic and classical
.rw


#END OF COMMANDS
.time
#END OF TIME
.stat
#END OF STATS


In [8]:
df = pd.read_json(json_path, lines=True)
#add a column called "is_pre_training" and set it to false
df["is_pretraining"] = False

pre_training_ids = ()
commands_file_path = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/38/pre_training/nyc_squad_pretraining_commands.txt"
with open(commands_file_path, 'r') as commands_file:
    lines = commands_file.readlines()
    for line in lines:
        if line.startswith("#id:"):
            pre_training_ids += (line.split("#id: ")[1].strip(),)
#mark the rows in the dataframe with the pre-training ids as is_pre_training = true
df.loc[df['id'].isin(pre_training_ids), 'is_pretraining'] = True
#write the df to a file called "nyc_squad_with_pretraining_flags.jsonl" in the same directory as the original json file
output_json_path = json_path.replace("nyc_squad_with_pretraining_commands.jsonl",
                                     "nyc_squad_with_pretraining_flags.jsonl")
df.to_json(output_json_path, orient='records', lines=True)
print(f"Output JSON with pre-training flags written to: {output_json_path}")

Output JSON with pre-training flags written to: /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/38/data_sets/nyc_squad_with_pretraining_flags.jsonl


In [2]:
from pipeline import Pipeline
import pandas as pd

pipeline = Pipeline(prepared_dataset_filepath=None)
base_path_mac = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD"
experiment_name = "49"
json_path = base_path_mac + f"/work/annabell/experiments/{experiment_name}/data_sets/squad_dataset_with_generated_sentences.jsonl"
pipeline.declarative_sentences_dataset = pd.read_json(json_path, lines=True)
pipeline.preprocess_dataset()
#pipeline.datasetPreProcessor.create_commands_for_pretraining()
#pipeline.run_pre_training_evaluation_testing()
#pipeline.run_evaluate_pre_training_results()
#pipeline.run_training()
#pipeline.run_testing()
#pipeline.run_evaluate_training_results()

2026-02-26 07:37:39,176 - pipeline - INFO - Starting dataset preprocessing...
2026-02-26 07:37:39,820 - dataset_processing - INFO - Checking if answer is in declarative statement for sample ID: 5727bec13acd2414000deb42
2026-02-26 07:37:39,821 - dataset_processing - INFO - Checking if answer is in declarative statement for sample ID: 5728a0da2ca10214002da4e9
2026-02-26 07:37:39,821 - dataset_processing - INFO - Checking if answer is in declarative statement for sample ID: 57281f423acd2414000df500
2026-02-26 07:37:39,822 - dataset_processing - INFO - Checking if answer is in declarative statement for sample ID: 56dff532231d4119001abf04
2026-02-26 07:37:39,822 - dataset_processing - INFO - Checking if answer is in declarative statement for sample ID: 57267b46dd62a815002e8695
2026-02-26 07:37:39,822 - dataset_processing - INFO - Answer 'a procession of people in animal disguise' not found in declarative statement: 'the effigy of nerthus was typically accompanied by a procession of people d

AttributeError: 'NoneType' object has no attribute 'lower'

In [3]:
#find the row with id = "56d4eb762ccc5a1400d8334f"
dataframe = pipeline.declarative_sentences_dataset
debug_df = dataframe[dataframe['id'] == "572ebc4803f98919007569b8"]
debug_df


,id,title,context,question,declarative_statement,answer
2230,572ebc4803f98919007569b8,Muammar_Gaddafi,"Many teachers at Sabha were Egyptian, and for ...",Name one event that occurred in the Arab world...,None,"1948 Arab–Israeli War, the Egyptian Revolution..."
